In [1]:
from IPython.display import display, HTML
display(HTML("<style>div.container { width:100% !important; }</style>"))

# 인공신경망 딥러닝 모델 구현

In [2]:
import tensorflow
tensorflow.__version__

'2.13.0'

## iris 데이터 불러오기

In [3]:
from sklearn import datasets

In [4]:
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target

In [5]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y, test_size=0.3,
                                                    random_state=1)

## 싸이킷런을 이용한 구현

In [25]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000) 

In [26]:
mlp.fit(train_X, train_y)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 30), max_iter=1000)

In [27]:
mlp.score(test_X, test_y)

1.0

In [9]:
pred = mlp.predict(test_X)

In [10]:
import pandas as pd
pd.crosstab(pred, test_y) #교차분류표
# pd.crosstab(np.argmax(pred, axis=1), np.argmax(test_y, axis=1)) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


## 텐서플로우를 이용한 구현

In [11]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])

In [13]:
W1 = tf.Variable(tf.zeros([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(x, W1)+b1)

In [14]:
W2 = tf.Variable(tf.zeros([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

In [15]:
W3 = tf.Variable(tf.zeros([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [16]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [17]:
import pandas as pd

feed_train = {x: train_X, y: pd.get_dummies(train_y).values}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print(i, tr_loss)

0 1.0978113
100 1.0611217
200 0.76452047
300 0.6687561
400 0.6138217
500 0.56832063
600 0.5274489
700 0.4911589
800 0.45574504
900 0.41874182


In [18]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {x: test_X, y: pd.get_dummies(test_y).values}
print(sess.run(accuracy, feed_dict=feed_test))

0.8888889


In [19]:
sess.close()

## 케라스를 이용한 구현

In [20]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [21]:
from tensorflow.keras.layers import InputLayer, Dense
model.add(InputLayer(4))
model.add(Dense(50, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [22]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [23]:
model.fit(train_X, train_y, epochs=200)

Train on 105 samples
Epoch 1/200
105/105 [==============================] - 0s 338us/sample - loss: 1.4009 - acc: 0.3048
Epoch 2/200
105/105 [==============================] - 0s 39us/sample - loss: 1.2432 - acc: 0.3048
Epoch 3/200
105/105 [==============================] - 0s 49us/sample - loss: 1.1182 - acc: 0.3048
Epoch 4/200
105/105 [==============================] - 0s 40us/sample - loss: 1.0337 - acc: 0.3048
Epoch 5/200
105/105 [==============================] - 0s 41us/sample - loss: 0.9724 - acc: 0.3238
Epoch 6/200
105/105 [==============================] - 0s 27us/sample - loss: 0.9192 - acc: 0.6286
Epoch 7/200
105/105 [==============================] - 0s 30us/sample - loss: 0.8769 - acc: 0.6857
Epoch 8/200
105/105 [==============================] - 0s 43us/sample - loss: 0.8381 - acc: 0.6952
Epoch 9/200
105/105 [==============================] - 0s 40us/sample - loss: 0.8052 - acc: 0.6952
Epoch 10/200
105/105 [==============================] - 0s 33us/sample - loss: 0.7769 -

105/105 [==============================] - 0s 35us/sample - loss: 0.1470 - acc: 0.9619
Epoch 83/200
105/105 [==============================] - 0s 35us/sample - loss: 0.1502 - acc: 0.9619
Epoch 84/200
105/105 [==============================] - 0s 24us/sample - loss: 0.1454 - acc: 0.9619
Epoch 85/200
105/105 [==============================] - 0s 23us/sample - loss: 0.1410 - acc: 0.9619
Epoch 86/200
105/105 [==============================] - 0s 26us/sample - loss: 0.1389 - acc: 0.9714
Epoch 87/200
105/105 [==============================] - 0s 29us/sample - loss: 0.1364 - acc: 0.9714
Epoch 88/200
105/105 [==============================] - 0s 36us/sample - loss: 0.1350 - acc: 0.9619
Epoch 89/200
105/105 [==============================] - 0s 31us/sample - loss: 0.1324 - acc: 0.9714
Epoch 90/200
105/105 [==============================] - 0s 30us/sample - loss: 0.1333 - acc: 0.9619
Epoch 91/200
105/105 [==============================] - 0s 29us/sample - loss: 0.1288 - acc: 0.9714
Epoch 92/200


105/105 [==============================] - 0s 40us/sample - loss: 0.1026 - acc: 0.9619
Epoch 164/200
105/105 [==============================] - 0s 31us/sample - loss: 0.0894 - acc: 0.9714
Epoch 165/200
105/105 [==============================] - 0s 41us/sample - loss: 0.0862 - acc: 0.9619
Epoch 166/200
105/105 [==============================] - 0s 35us/sample - loss: 0.0920 - acc: 0.9619
Epoch 167/200
105/105 [==============================] - 0s 28us/sample - loss: 0.0896 - acc: 0.9619
Epoch 168/200
105/105 [==============================] - 0s 35us/sample - loss: 0.0855 - acc: 0.9714
Epoch 169/200
105/105 [==============================] - 0s 36us/sample - loss: 0.0920 - acc: 0.9714
Epoch 170/200
105/105 [==============================] - 0s 33us/sample - loss: 0.0907 - acc: 0.9714
Epoch 171/200
105/105 [==============================] - 0s 32us/sample - loss: 0.0845 - acc: 0.9714
Epoch 172/200
105/105 [==============================] - 0s 34us/sample - loss: 0.0845 - acc: 0.9714
Epoc

In [28]:
model.evaluate(test_X, test_y) # 출력값은 loss와 accuracy

[0.0609300847682688, 1.0]